<a target="_blank" href="https://colab.research.google.com/github/sonder-art/fdd_o23/blob/main/codigo/polars/04_ejercicio_polars.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [17]:
#!pip install polars

In [18]:
import polars as pl
import random
import string


# Generacion de Datos

In [19]:
# Funciones auxiliares para generar datos
def random_string(length=5):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

def generate_data(n=1000):
    data = {
        "id_venta": list(range(1, n + 1)),
        "producto": [random_string() for _ in range(n)],
        "cantidad": [random.randint(1, 20) for _ in range(n)],
        "precio_unitario": [random.uniform(10, 100) for _ in range(n)],
        "fecha_venta": [pl.date(random.randint(2010, 2022), random.randint(1, 12), random.randint(1, 28)) for _ in range(n)],
        "cliente_id": [random.randint(1000, 5000) for _ in range(n)]
    }
    return pl.DataFrame(data)

# Generar el DataFrame
df = generate_data()
df.head()

id_venta,producto,cantidad,precio_unitario,fecha_venta,cliente_id
i64,str,i64,f64,object,i64
1,"""drydt""",13,54.404756,"2013-05-23 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",4350
2,"""mvxuk""",18,98.174145,"2022-07-24 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",4445
3,"""ksrpz""",9,69.726795,"2016-05-02 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",4744
4,"""ztdoi""",2,85.910555,"2021-01-26 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",1191
5,"""vmsfj""",18,23.469495,"2010-04-24 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",2402


In [20]:
df = df.lazy()
df

# Limpieza de Datos

Utilizando Polars en modo Lazy, realice las siguientes operaciones de limpieza y manipulación de datos:

    1. `Filtrar` las ventas con cantidades menores a 5.
    2. `Agregar` una nueva columna llamada total_venta, que sea el resultado de multiplicar cantidad por precio_unitario.
    3. `Eliminar` la columna precio_unitario después de crear total_venta.
    4. `Agrupar` los datos por producto y calcular el total vendido y la cantidad total vendida por producto.
    5. `Ordenar` los resultados por el total vendido en orden descendente.

Todo esto lo debes de realizar en una sola linea de codigo

In [21]:

lazy_df = df.lazy()

# Realizar las operaciones de limpieza y manipulación

resultado = (
    lazy_df
    .filter(pl.col("cantidad")<5)
    .with_columns([(pl.col("cantidad")*pl.col("precio_unitario")).alias("total_venta")])
    .drop("precio_unitario")
    .group_by("producto").agg([
        pl.col("total_venta").sum().alias("total_vendido"),
        pl.col("cantidad").sum().alias("cantidad_total_vendida")
    ])
    .sort("total_vendido")

)

# Ejecutar y mostrar los resultados
resultado_df = resultado.collect()
print(resultado_df)


shape: (219, 3)
┌──────────┬───────────────┬────────────────────────┐
│ producto ┆ total_vendido ┆ cantidad_total_vendida │
│ ---      ┆ ---           ┆ ---                    │
│ str      ┆ f64           ┆ i64                    │
╞══════════╪═══════════════╪════════════════════════╡
│ wopmo    ┆ 10.611182     ┆ 1                      │
│ sbiiq    ┆ 11.857285     ┆ 1                      │
│ jlolk    ┆ 15.967401     ┆ 1                      │
│ wpffr    ┆ 20.501723     ┆ 1                      │
│ oweij    ┆ 21.666265     ┆ 1                      │
│ …        ┆ …             ┆ …                      │
│ okgro    ┆ 379.694083    ┆ 4                      │
│ sxuxa    ┆ 381.413264    ┆ 4                      │
│ upakp    ┆ 384.601241    ┆ 4                      │
│ sphjc    ┆ 384.63735     ┆ 4                      │
│ uital    ┆ 393.096908    ┆ 4                      │
└──────────┴───────────────┴────────────────────────┘


# Guarda el resultado en un archivo `parquet`

Guarda el resultado en un archivo parquet que se llame `resultados.parquet`

In [22]:
# Tu codigo aqui
# Save the DataFrame to a Parquet file
resultado_df.write_parquet("resultados.parquet")

# Lee Parquet

Crea codigo para leer en polars el archivo parquet

In [23]:
# Tu codigo aqui
# Read the Parquet file into a Polars DataFrame
df = pl.read_parquet("resultados.parquet")

# Display the DataFrame
print(df)

shape: (219, 3)
┌──────────┬───────────────┬────────────────────────┐
│ producto ┆ total_vendido ┆ cantidad_total_vendida │
│ ---      ┆ ---           ┆ ---                    │
│ str      ┆ f64           ┆ i64                    │
╞══════════╪═══════════════╪════════════════════════╡
│ wopmo    ┆ 10.611182     ┆ 1                      │
│ sbiiq    ┆ 11.857285     ┆ 1                      │
│ jlolk    ┆ 15.967401     ┆ 1                      │
│ wpffr    ┆ 20.501723     ┆ 1                      │
│ oweij    ┆ 21.666265     ┆ 1                      │
│ …        ┆ …             ┆ …                      │
│ okgro    ┆ 379.694083    ┆ 4                      │
│ sxuxa    ┆ 381.413264    ┆ 4                      │
│ upakp    ┆ 384.601241    ┆ 4                      │
│ sphjc    ┆ 384.63735     ┆ 4                      │
│ uital    ┆ 393.096908    ┆ 4                      │
└──────────┴───────────────┴────────────────────────┘
